Critique du film IRobot sur Allociné
http://www.allocine.fr/film/fichefilm-47739/critiques/spectateurs/

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
# URL
url = 'http://www.allocine.fr/film/fichefilm-47739/critiques/spectateurs/'
# URI-SUFFIX
uri_pages = '?page='
# Number of pages to read
nbPages = 61
# XPath content to collect
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]' ]
cols = ['Note', \
        'Description' ]

In [3]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [4]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

In [5]:
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

liste_totale = getPages(nbPages, url)
liste_totale.head()

,Note,Description,key
0,"4,0","Un univers original dans ce film, Will Smith j...",0
1,"4,0",Un très bon film de science fiction . L'histoi...,1
2,"4,0",après avoir longtemps entendu parler de ce fil...,2
3,"4,0",Un film de science-fiction qui emprunte beauco...,3
4,"4,0",le dernier bébé de Proyas en date.inspiré dire...,4


In [6]:
liste_totale.to_csv('../datasources/films/allocine_irobot_avis.csv', index=False)